In [1]:
%%writefile app.py
import streamlit as st
import torch
import plotly.express as px
from transformers import BertTokenizer, BertForSequenceClassification

st.set_page_config(page_title="Balanced Sentiment AI", page_icon="🎬")

@st.cache_resource
def load_bert():
    try:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained(".")
        model.eval()
        return tokenizer, model
    except Exception as e:
        return str(e), None

st.title("🎬 IMDb Sentiment Analyzer Dashboard")

tokenizer, model = load_bert()

user_input = st.text_area(" Please Enter your Review:")

if st.button("Predict"):
    if user_input:
        inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=128)
        
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            
            # --- MODERATE TEMPERATURE ---
            temperature = 2.0  # Balance scaling
            probs = torch.nn.functional.softmax(logits / temperature, dim=-1)
            
            neg_score = probs[0][0].item()
            pos_score = probs[0][1].item()
            
        st.divider()

        # --- REFINED LOGIC ---
        # show Mixed when both  weights  between 15% - 85% 
        if 0.15 <= pos_score <= 0.85:
            result = "MIXED SENTIMENT 😐"
            color = "orange"
        elif pos_score > 0.65:
            result = "POSITIVE 😃"
            color = "green"
        else:
            result = "NEGATIVE 😞"
            color = "red"

        st.markdown(f"### Result: :{color}[{result}]")
        
        col1, col2 = st.columns(2)
        
        with col1:
            st.metric("Positive", f"{pos_score*100:.1f}%")
            st.metric("Negative", f"{neg_score*100:.2f}%")
        
        with col2:
            # --- PIE CHART GRAPH ---
            fig = px.pie(
                values=[pos_score, neg_score], 
                names=['Positive', 'Negative'],
                color=['Positive', 'Negative'],
                color_discrete_map={'Positive':'#2ecc71', 'Negative':'#e74c3c'},
                hole=0.4
            )
            fig.update_layout(margin=dict(t=0, b=0, l=0, r=0), height=220)
            st.plotly_chart(fig, use_container_width=True)
            
        st.progress(pos_score)

Overwriting app.py
